In [8]:
! pip install mlflow boto3 awscli optuna xgboost imbalanced-learn
! pip install optuna

In [9]:
import mlflow
# step 1 : Set up the mlflow tracking server 
mlflow.set_tracking_uri("http://ec2-13-62-127-174.eu-north-1.compute.amazonaws.com:5000/")

In [10]:
# Set or create an experiment
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

<Experiment: artifact_location='s3://reddit-reccomender-bucket/11', creation_time=1763396487435, experiment_id='11', last_update_time=1763396487435, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [11]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import RandomUnderSampler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [12]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [13]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for Logistic regression

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

## Step 2: Train-test split BEFORE sampling

X_train_raw,X_test_raw,y_train,y_test = train_test_split(
        df['clean_comment'],df['category'],
        test_size=0.2,
        random_state=42,
        stratify=df['category']
)

# Step 3: BOW vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000

vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=max_features)

X_train_vec = vectorizer.fit_transform(X_train_raw)
X_test_vec = vectorizer.transform(X_test_raw)

# Step 4: Apply Random Undersampler to train only
rus = RandomUnderSampler(random_state=42)
X_train_res, y_train_res = rus.fit_resample(X_train_vec, y_train)



#Function to log results in MLflow
def log_mlflow(model_name,model,X_train,X_test,y_train,y_test):
    with mlflow.start_run():
        #log the model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_Undersampling_BOW_Trigram")
        mlflow.set_tag("experiment_type", "Algorithm comparision")

        #log algorith name as parameter
        mlflow.log_param("algo_name",model_name)

        #Train model
        model.fit(X_train,y_train)
        y_pred =model.predict(X_test)

        # log accuracy
        accuracy = accuracy_score(y_test,y_pred)
        mlflow.log_metric("accuracy",accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for Logistic regression
def objective_lorreg(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])

    model = LogisticRegression(C=C, penalty=penalty, solver='liblinear', random_state=42)
    model.fit(X_train_res, y_train_res)
    preds = model.predict(X_test_vec)

    return accuracy_score(y_test,preds)


# Step 7: Run Optuna for LOR, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_lorreg, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = LogisticRegression(C=best_params['C'], penalty=best_params['penalty'], solver='liblinear', random_state=42)

    # Log the best model with MLflow, passing the algo_name as "LOR"
    log_mlflow("LOR", best_model, X_train_res, X_test_vec, y_train_res, y_test)

# Run the experiment for LOR
run_optuna_experiment()




[I 2025-11-18 12:07:40,230] A new study created in memory with name: no-name-9254e01a-8e6f-4e00-b2fd-ca7d811ad51c
c:\Users\mukti\Desktop\python_DS\campus_x\reddit_sentiment_analysis\aws_env\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
[I 2025-11-18 12:07:40,412] Trial 0 finished with value: 0.22501022773762444 and parameters: {'C': 0.0002309112791330923, 'penalty': 'l1'}. Best is trial 0 with value: 0.22501022773762444.
c:\Users\mukti\Desktop\python_DS\campus_x\reddit_sentiment_analysis\aws_env\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another

🏃 View run LOR_Undersampling_BOW_Trigram at: http://ec2-13-62-127-174.eu-north-1.compute.amazonaws.com:5000/#/experiments/11/runs/9dac577a60b14495b576bbc455139cfe
🧪 View experiment at: http://ec2-13-62-127-174.eu-north-1.compute.amazonaws.com:5000/#/experiments/11
